In [ ]:
""" This code allows to identify El Niño and La Niña years, extract them and make a composite with all the El Niño and La Niña events together

This particular example defines ENSO events in the Niño3.4 region

Written by Paloma Trascasa-Castro

"""

In [ ]:
# ENSO composites

## This code allows to identify El Niño and La Niña years, extract them and make a composite with all the El Niño and La Niña events together

## This particular example defines ENSO events in the Niño3.4 region

## Written by Paloma Trascasa-Castro



In [ ]:
# Import some libraries:

%matplotlib inline
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import iris
import matplotlib.cm as mpl_cm
import iris.coord_categorisation as icc
from numpy import meshgrid 
import warnings
from iris.cube import Cube
from iris.coords import DimCoord


In [ ]:
# 1) Load your sea surface temperature data as an Iris cube:

sst=iris.load_cube('/path/file.nc')


# 2) There are many ways to identify ENSO events, here we'll use the Niño3.4 region (5N - 5S, 170W - 120W)
#    as this is the most common appoach 

# Extract the area and average over the latitude and longitude dimensions

sst=sst.intersection(longitude=(-170,-120))
sst=sst.intersection(latitude=(-5,5))
sst=sst.collapsed('longitude',iris.analysis.MEAN)
sst=sst.collapsed('latitude',iris.analysis.MEAN)

# Calculate SST anomalies over time

sst_mean=sst.collapsed('time',iris.analysis.MEAN)
sst_an=sst-sst_mean.data

# 3) Threshold: +0.5C for El Niño and -0.5C for La Niña. This identifies the position (indices) of El Niño / La Niña events in your timeseries

threshold=0.5

# El Niño = SST anomalies are higher than 0.5C. 
nino_years=np.nonzero(sst_an.data > thres)

# La Niña = SST anomalies are lower than -0.5C
nina_years=np.nonzero(sst_an.data < -thres)


# 4) Extract El Niño and La Niña years to make composites:

nino=sst[nino_years]
nina=sst[nina_years]

# Now you have 2 cubes one for each ENSO phase, with as many time points as El Niño and La Niña events


In [ ]:
# Plot

# Average over time
nino_tmean=nino.collapsed('time',iris.analysis.MEAN)

plt.figure(figsize=(6, 2.5),dpi=350)

# Choose your colour palette, in this case I use the "RdBu_r" that goes from blue to red
brewer_cmap = mpl_cm.get_cmap('RdBu_r',50)

# Restrict your area to the tropical Pacific
m = Basemap(projection='gall',llcrnrlat=-30,urcrnrlat=30,\
            llcrnrlon=120,urcrnrlon=300,resolution='c')

lat=nino_tmean.coord('latitude').points
lon=nino_tmean.coord('longitude').points

levels=np.linspace(-2,2,21)
xx, yy = m(*np.meshgrid(lon,lat))

# Plot
cs=m.contourf(xx,yy,nino_tmean.data,levels=levels,cmap=brewer_cmap,extend="both") 

# Add some feautres: Coastlines, parallels, meridians, colorbar, labels, etc.

m.drawcoastlines(linewidth=0.8)
m.drawparallels(np.arange(-90.,91.,10.), labels=[1,0,0,0], fontsize = 7,linewidth=0.5)    
m.drawmeridians(np.arange(0.,361.,20.), labels=[1,1,0,1], fontsize = 7,linewidth=0.5)
cbar=plt.colorbar(cs,orientation="horizontal",aspect=40,pad=0.1,ticks=[-2,-1.5,-1,-0.5,0,0.5,1,1.5,2])
m.fillcontinents(color='black',lake_color='aqua')
cbar.set_label('SST anomalies (K)',size=15)
cbar.ax.tick_params(labelsize=7) 
plt.title('El Niño',loc="center",fontsize=12)

# Save your plot

plt.savefig('/path/elninosstpattern.png',bbox_inches = "tight", dpi=350)
plt.show()